### Import the needed packages

In [1]:
# package for using the the custom scripts 
import sys
sys.path.insert(0, "./../")
# package for handling files and directories
import os
# package for handling the training hyperparameter tracking
import wandb
# a custom package for handling the data
import utilities as utils
# package for handling the versioning of the models
from huggingface_hub import HfApi
# package for handling the environment variables
from dotenv import load_dotenv
# get the environment variables
load_dotenv()
# set the wandb api to silent
os.environ["WANDB_SILENT"] = "true"

c:\Users\Acer\miniconda3\envs\tf-gpu\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Users\Acer\miniconda3\envs\tf-gpu\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If 

### Global variable

In [2]:
# store the path
path_full = os.environ.get("ORI_PATH")
path_dataset_ori = os.path.join(path_full, "datasets", "splitted", "original")
path_dataset_aug = os.path.join(path_full, "datasets", "splitted", "augmented")
path_model = os.path.join(path_full, "data", "model")
ori_path = {
    "train": os.path.join(path_dataset_ori, "train"),
    "val": os.path.join(path_dataset_ori, "val"),
    "test": os.path.join(path_dataset_ori, "test"),
}
aug_path = {
    "train": os.path.join(path_dataset_aug, "train"),
    "val": os.path.join(path_dataset_aug, "val"),
    "test": os.path.join(path_dataset_aug, "test"),
}
# the commit message
wandb_init_name = "retrain the model"
hf_commit_message = "retrain the model"
# the hyperparameter for the training
hyperparam = {
    "batch_size": 16,
    "epochs": 50,
    "img_size": 128,
    "shuffle": True,
    "model_type": "U-net",
    "metrics": ["accuracy", "MeanIoU", "Precision", "Recall"],
    "split_ratio": [7,1,2],
    "dataset_balance": True,
    "weighted_training": True,
}

### Import the images and mask

In [3]:
# get the original dataset
train_ori_img, train_ori_mask = utils.load_img_mask(ori_path["train"])
val_ori_img, val_ori_mask = utils.load_img_mask(ori_path["val"])
test_ori_img, test_ori_mask = utils.load_img_mask(ori_path["test"])

# get the augmented dataset
train_aug_img, train_aug_mask = utils.load_img_mask(aug_path["train"])
val_aug_img, val_aug_mask = utils.load_img_mask(aug_path["val"])
test_aug_img, test_aug_mask = utils.load_img_mask(aug_path["test"])

### Create the batched dataset

In [4]:
# create the tensor dataset for original dataset
trainset_ori = utils.create_dataset(train_ori_img, train_ori_mask, hyperparam["img_size"], hyperparam["batch_size"])
valset_ori = utils.create_dataset(val_ori_img, val_ori_mask, hyperparam["img_size"], hyperparam["batch_size"])
testset_ori = utils.create_dataset(test_ori_img, test_ori_mask, hyperparam["img_size"], hyperparam["batch_size"])

# create the tensor dataset for augmented dataset
trainset_aug = utils.create_dataset(train_aug_img, train_aug_mask, hyperparam["img_size"], hyperparam["batch_size"])
valset_aug = utils.create_dataset(val_aug_img, val_aug_mask, hyperparam["img_size"], hyperparam["batch_size"])
testset_aug = utils.create_dataset(test_aug_img, test_aug_mask, hyperparam["img_size"], hyperparam["batch_size"])

### Train model

### Unet

In [ ]:
# create the unet model
unet_custom = utils.custom_unet()
file_model_name = "unet_model_ori"
# train the model using original dataset
unet_custom, history, ev_result = utils.train_model(unet_custom,
                                                trainset_ori, valset_ori, testset_ori,
                                                path_model, file_model_name,
                                                hyperparam["epochs"])
# show the evaluation result
print(f"test loss: {ev_result[0]}", f"test mean_px_acc: {ev_result[1]}",
        f"test auc: {ev_result[2]}", f"test precision: {ev_result[3]}",
        f"test recall: {ev_result[4]}", sep="\n")

In [ ]:
# initiate the huggingface api 
api = HfApi()
repo = api.create_repo("CAD-Glaucoma_Segmentation", repo_type="model", exist_ok=True)

In [ ]:
# push the model artifact and hyperparameter to wandb
## set the hyperparameter for the wandb
hyperparam["aug"] = False
## initiate the project
wandb.init(project="CAD - Glaucoma Segmentation",
            name=f"ori - {wandb_init_name}",
            config=hyperparam)
## write the training history and validation result to wandb log
for epoch in range(len(history.history["loss"])):
    wandb.log({
        "train_loss": history.history["loss"][epoch],
        "train_accuracy": history.history["mean_px_acc"][epoch],
        "train_auc": history.history["auc"][epoch],
        "train_precision": history.history["precision"][epoch],
        "train_recall": history.history["recall"][epoch],
        "train_f1": 2 * (history.history["precision"][epoch] * history.history["recall"][epoch]) / (history.history["precision"][epoch] + history.history["recall"][epoch] + 1e-7),
        "val_loss": history.history["val_loss"][epoch],
        "val_accuracy": history.history["val_mean_px_acc"][epoch],
        "val_auc": history.history["val_auc"][epoch],
        "val_precision": history.history["val_precision"][epoch],
        "val_recall": history.history["val_recall"][epoch],
        "val_f1": 2 * (history.history["val_precision"][epoch] * history.history["val_recall"][epoch]) / (history.history["val_precision"][epoch] + history.history["val_recall"][epoch] + 1e-7),
    })
## store the model artifact
wandb.log_artifact(os.path.join(path_model, f"{file_model_name}.h5"))
wandb.finish()

In [ ]:
# push the model to the huggingface hub
api.upload_file(
    repo_id=repo.repo_id,
    path_or_fileobj=os.path.join(path_model, f"{file_model_name}.h5"),
    path_in_repo=f"model/{file_model_name}.h5",
    repo_type=repo.repo_type,
    commit_message=hf_commit_message,
)

In [ ]:
# create the unet model
unet_custom = utils.custom_unet()
file_model_name = "unet_model_aug"
# train the model using augmented dataset
unet_custom, history, ev_result = utils.train_model(unet_custom,
                                                trainset_aug, valset_aug, testset_aug,
                                                path_model, file_model_name,
                                                hyperparam["epochs"])
# show the evaluation result 
print(f"test loss: {ev_result[0]}", f"test mean_px_acc: {ev_result[1]}",
        f"test auc: {ev_result[2]}", f"test precision: {ev_result[3]}",
        f"test recall: {ev_result[4]}", sep="\n")

In [ ]:
# push the model artifact and hyperparameter to wandb
## set the hyperparameter for the wandb
hyperparam["aug"] = True
## initiate the project
wandb.init(project="CAD - Glaucoma Segmentation",
            name=f"aug - {wandb_init_name}",
            config=hyperparam)
## write the training history and validation result to wandb log
for epoch in range(len(history.history["loss"])):
    wandb.log({
        "train_loss": history.history["loss"][epoch],
        "train_accuracy": history.history["mean_px_acc"][epoch],
        "train_auc": history.history["auc"][epoch],
        "train_precision": history.history["precision"][epoch],
        "train_recall": history.history["recall"][epoch],
        "train_f1": 2 * (history.history["precision"][epoch] * history.history["recall"][epoch]) / (history.history["precision"][epoch] + history.history["recall"][epoch] + 1e-7),
        "val_loss": history.history["val_loss"][epoch],
        "val_accuracy": history.history["val_mean_px_acc"][epoch],
        "val_auc": history.history["val_auc"][epoch],
        "val_precision": history.history["val_precision"][epoch],
        "val_recall": history.history["val_recall"][epoch],
        "val_f1": 2 * (history.history["val_precision"][epoch] * history.history["val_recall"][epoch]) / (history.history["val_precision"][epoch] + history.history["val_recall"][epoch] + 1e-7),
    })
## store the model artifact
wandb.log_artifact(os.path.join(path_model, f"{file_model_name}.h5"))
wandb.finish()

In [ ]:
## push the model to the huggingface hub
api.upload_file(
    repo_id=repo.repo_id,
    path_or_fileobj=os.path.join(path_model, f"{file_model_name}.h5"),
    path_in_repo=f"model/{file_model_name}.h5",
    repo_type=repo.repo_type,
    commit_message=f"aug - {hf_commit_message}",
)

### MobileNet

In [ ]:
# create the unet model
mobileNet_ori = utils.mobilenet_model()
file_model_name = "mobileNet_model_ori"
# train the model using original dataset
mobileNet_ori, history, ev_result = utils.train_model(mobileNet_ori,
                                                trainset_ori, valset_ori, testset_ori,
                                                path_model, file_model_name,
                                                hyperparam["epochs"])
# show the evaluation result
print(f"test loss: {ev_result[0]}", f"test mean_px_acc: {ev_result[1]}",
        f"test auc: {ev_result[2]}", f"test precision: {ev_result[3]}",
        f"test recall: {ev_result[4]}", sep="\n")

In [ ]:
mobileNet_ori.save(f"{file_model_name}.h5")

In [ ]:
# create the unet model
mobileNet_aug = utils.mobilenet_model()
file_model_name = "mobileNet_model_aug"
# train the model using augmented dataset
mobileNet_aug, history, ev_result = utils.train_model(mobileNet_aug,
                                                trainset_aug, valset_aug, testset_aug,
                                                path_model, file_model_name,
                                                hyperparam["epochs"])
# show the evaluation result
print(f"test loss: {ev_result[0]}", f"test mean_px_acc: {ev_result[1]}",
        f"test auc: {ev_result[2]}", f"test precision: {ev_result[3]}",
        f"test recall: {ev_result[4]}", sep="\n")

In [ ]:
mobileNet_aug.save(f"{file_model_name}.h5")

### EfficientNet

In [5]:
# create the unet model
efficientNet_ori = utils.efficientNet_model()
file_model_name = "efficientNet_model_ori"
# train the model using original dataset
efficientNet_ori, history, ev_result = utils.train_model(efficientNet_ori,
                                                trainset_ori, valset_ori, testset_ori,
                                                path_model, file_model_name,
                                                hyperparam["epochs"])
# show the evaluation result
print(f"test loss: {ev_result[0]}", f"test mean_px_acc: {ev_result[1]}",
        f"test auc: {ev_result[2]}", f"test precision: {ev_result[3]}",
        f"test recall: {ev_result[4]}", sep="\n")

Epoch 1/50
16/16 [==============================] - 34s 546ms/step - loss: 0.0136 - mean_px_acc: 0.3853 - auc: 0.6458 - precision: 0.4720 - recall: 0.3968 - val_loss: 0.0128 - val_mean_px_acc: 0.0659 - val_auc: 0.6374 - val_precision: 0.4446 - val_recall: 0.4158
Epoch 2/50
16/16 [==============================] - 6s 143ms/step - loss: 0.0097 - mean_px_acc: 0.5144 - auc: 0.7441 - precision: 0.5615 - recall: 0.4257 - val_loss: 0.0130 - val_mean_px_acc: 0.0857 - val_auc: 0.6475 - val_precision: 0.4645 - val_recall: 0.4012
Epoch 3/50
16/16 [==============================] - 5s 139ms/step - loss: 0.0087 - mean_px_acc: 0.6674 - auc: 0.7901 - precision: 0.6172 - recall: 0.4565 - val_loss: 0.0101 - val_mean_px_acc: 0.3040 - val_auc: 0.7039 - val_precision: 0.5319 - val_recall: 0.3604
Epoch 4/50
16/16 [==============================] - 5s 129ms/step - loss: 0.0082 - mean_px_acc: 0.7697 - auc: 0.8149 - precision: 0.6424 - recall: 0.4944 - val_loss: 0.0087 - val_mean_px_acc: 0.6288 - val_auc: 0.7

In [9]:
# efficientNet_ori.save(f"{file_model_name}.h5", save_format="h5")

import json

try:
    efficientNet_ori.save("my_model.h5", save_format="h5")
except TypeError as e:
    print("Serialization Error:", str(e))
    # Simpan struktur model untuk investigasi
    with open("model_structure.json", "w") as f:
        f.write(efficientNet_ori.to_json())

Serialization Error: Unable to serialize [2.0896919 2.1128857 2.1081853] to JSON. Unrecognized type <class 'tensorflow.python.framework.ops.EagerTensor'>.


TypeError: Unable to serialize [2.0896919 2.1128857 2.1081853] to JSON. Unrecognized type <class 'tensorflow.python.framework.ops.EagerTensor'>.

In [11]:
# create the unet model
efficientNet_aug = utils.efficientNet_model()
file_model_name = "efficientNet_model_aug"
# train the model using original dataset
efficientNet_aug, history, ev_result = utils.train_model(efficientNet_aug,
                                                trainset_aug, valset_aug, testset_aug,
                                                path_model, file_model_name,
                                                hyperparam["epochs"])
# show the evaluation result
print(f"test loss: {ev_result[0]}", f"test mean_px_acc: {ev_result[1]}",
        f"test auc: {ev_result[2]}", f"test precision: {ev_result[3]}",
        f"test recall: {ev_result[4]}", sep="\n")

Epoch 1/50
16/16 [==============================] - 17s 276ms/step - loss: 0.0142 - mean_px_acc: 0.3501 - auc: 0.6256 - precision: 0.4375 - recall: 0.3765 - val_loss: 0.0167 - val_mean_px_acc: 0.0898 - val_auc: 0.6364 - val_precision: 0.4336 - val_recall: 0.4255
Epoch 2/50
16/16 [==============================] - 3s 115ms/step - loss: 0.0103 - mean_px_acc: 0.4170 - auc: 0.7206 - precision: 0.5266 - recall: 0.4424 - val_loss: 0.0155 - val_mean_px_acc: 0.0749 - val_auc: 0.6272 - val_precision: 0.4291 - val_recall: 0.4089
Epoch 3/50
16/16 [==============================] - 3s 110ms/step - loss: 0.0093 - mean_px_acc: 0.4883 - auc: 0.7542 - precision: 0.5749 - recall: 0.4391 - val_loss: 0.0133 - val_mean_px_acc: 0.0674 - val_auc: 0.6301 - val_precision: 0.4331 - val_recall: 0.4012
Epoch 4/50
16/16 [==============================] - 3s 111ms/step - loss: 0.0087 - mean_px_acc: 0.5809 - auc: 0.7784 - precision: 0.6051 - recall: 0.4393 - val_loss: 0.0096 - val_mean_px_acc: 0.3295 - val_auc: 0.6

In [12]:
efficientNet_aug.save(f"{file_model_name}.h5")

TypeError: Unable to serialize [2.0896919 2.1128857 2.1081853] to JSON. Unrecognized type <class 'tensorflow.python.framework.ops.EagerTensor'>.